# Geodatenanalyse 1

## Tag 4 / Block 2 / Übung 10: Deterministische Interpolationsverfahren

Folgende Python Packages, bzw. Libraries enthalten verschiedene, nützliche Funktionen für die Interpolation von räumlichen Daten: 

- scipy.interpolate: Univariate & Multivariate Interpolation, Funktionen für lineare, Spline, Nearest Neighbour und Radiale Basisfunktion Interpolation

- scipy.spatial: Delauney Tesselation, Voronoi Diagramme und plotting Funktionen

- Inverse Distance Weighting: z.B. über PyGeM (https://mathlab.github.io/PyGeM/index.html), oder andere Funktionen (https://github.com/paulbrodersen/inverse_distance_weighting)

In dieser Übung werden wir uns die Funktionen in `scipy` näher anschauen. Als Beispieldatensatz dienen gemessene Grundwasserparameter (Temperatur und Höhe) in Karlsruhe (Menberg et al., 2013). 

### 1. Explorative Datenanalyse

Ladet diese Daten aus "interp_data.csv" zunächst ein, und benutzt `DataFrame.describe()`, um einen ersten Überblick über die Daten und deren Werte zu bekommen. 

In [ ]:
import pandas as pd
data = pd.read_csv('interp_data.csv', sep=';', encoding='cp1252')

data.describe()

Um einen ersten Eindruck von der räumlichen Verteilung der Datenpunkte und ihrer Werte zu erhalten, erzeugt einen Scatterplot mit den Rechts- und Hochwerten, und der Grundwassertemperatur als Farbe der Punkte. 

Wählt dafür eine geeignete Farbskala ("cmap=...") in `matplotlib`. Eine Übersicht über die verfügbaren Farbskalen gibt es hier: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(1,1,1)

plt.scatter(data.Rechtswert, data.Hochwert, c=data.GWT, cmap='RdYlGn_r')
plt.xlabel('Rechtswert [m]')
plt.ylabel('Hochwert [m]')
plt.title('Grundwassertemperatur [°C]')
plt.grid(c='k', alpha=0.2)
plt.show()

### 2. scipy.spatial und scipy.interpolate.griddata

Für Funktionen in diesen Libraries benötigt Ihr die Koordinaten der Datenpunkte in Form eines Numpy Arrays mit zwei Spalten. Nach der Umwandlung könnt Ihr über `scipy.spatial.Delaunay()` ein Objekt mit einer Delaunay-Triangulation erzeugen. 

Plottet im Anschluss die Delaunay-Triangulation und die Lage der Datenpunkte. Benutzt für die Delaunay-Dreiecke `matplotlib.pyplot.triplot()`, mit den Koordinaten der Punkte als Inputs. Wenn kein Triangulations-Objekt als Input angegeben wird automatisch eine Delaunay-Triangulation dargestellt.

In [ ]:
from scipy.spatial import Delaunay

points = data[['Rechtswert', 'Hochwert']].values
tri = Delaunay(points)

plt.figure(5, figsize=(10,6))
plt.triplot(data.Rechtswert, data.Hochwert)
plt.plot(data.Rechtswert, data.Hochwert, "or")
plt.grid()
plt.xlabel('Rechtswert [m]')
plt.ylabel('Hochwert [m]')
plt.show()

Als nächtest wollen wir ein regelmäßiges Grid von Datenpunkten erzeugen, das die Ausgabepunkte der Interpolation darstellt. 

Definiert zuerst die Ausdehnung des Bereichs (xmin, max, ymin, ymax). Der Bereich sollte dabei den durch die Messpunkte abgedeckten Bereich umschließen, zuzüglich eines kleinen Puffers an den Ränern. Erstellt dann zwei Arrays für die x- und y-Koordinaten der Gridpunkte. Benutzt dafür die Funktion `np.mgrid[xmin:xmax:xstep, ymin:ymax:ystep]`. 

Plottet anschließend die Grid-Punkte, um die räumliche Verteilung zu inspizieren. 

In [ ]:
import numpy as np
extent = x_min, x_max, y_min, y_max = [data.Rechtswert.min()-1000, data.Rechtswert.max()+1000,
                                       data.Hochwert.min()-1000, data.Hochwert.max()+1000]
grid_x, grid_y = np.mgrid[x_min:x_max:400, y_min:y_max:400]

plt.figure(2, figsize=(8,6))
plt.scatter(grid_x, grid_y, s=10)
plt.xlabel('Rechtswert [m]')
plt.ylabel('Hochwert [m]')
plt.show()

Nun werden wir die Temperaturdaten mit scipy.interpolate.griddata() interpolieren. Als Inputs werden dafür die Koordinaten (als Numpy Array) und die zu interpolierenden Werte, sowie das Grid benötigt. scipy.interpolate.griddata() unterstützt verschiedene Interpolationsfunktionen, die über den Input method=... spezifizert werden können: "nearest" für Nearest Neighbour, "linear" für eine einfache lineare Interpolation, und "cubic" für eine kubische Spline-Interpolation.

Testet alle drei Methoden mit Hilfe des oben erzeugten Grids, und plottet sie zum Vergleich nebeneinander. Zum Plotten von Rasterdaten eignet sich matplotlib.pyplot.imshow(), das als Inputs das darzustellende Raster (mit .T transponiert), den Ausgangspunkt (origin='lower') und die Randkoordinaten des Rasters benötigt.

Wie gefallen Euch die Ergebnisse der Interpolation?

In [ ]:
from scipy.interpolate import griddata

grid_z0 = griddata(points, data.GWT.values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, data.GWT.values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, data.GWT.values, (grid_x, grid_y), method='cubic')

fig, axs = plt.subplots(ncols=3, figsize=(15, 5))

ax = axs[0]
ax.imshow(grid_z0.T, origin='lower', extent=extent, cmap='RdYlGn_r')
ax.scatter(data.Rechtswert, data.Hochwert, s=2, c='w')
ax.set_title('Nearest Neighbour')

ax = axs[1]
ax.imshow(grid_z1.T, origin='lower', extent=extent, cmap='RdYlGn_r')
ax.scatter(data.Rechtswert, data.Hochwert, s=2, c='w')
ax.set_title('Lineare Interpolation')

ax = axs[2]
ax.imshow(grid_z2.T, origin='lower', extent=extent, cmap='RdYlGn_r')
ax.scatter(data.Rechtswert, data.Hochwert, s=2, c='w')
ax.set_title('Kubischer Spline')

plt.show()

### 3. scipy.interpolate.Rbf

Neben den Interpolationsmethoden oben, gibt es in scipy auch verschiedenen Möglichkeiten zur Interpolation mit Radialen Basisfunktionen (RBFs). Die default-Einstellung von `scipy.interpolate.Rbf()` ist dabei eine Multi-quadratische Funktion, die exakt durch die Stützpunkte geht (exakte Interpolation). 

Definiert nun in dem nächsten Skript zuerst ein Interpolationsobjekt mit Hilfe von `scipy.interpolate.Rbf()`, und den x- und y-Koordinaten Eurer Messpunkte, sowie den Messwerten als Inputs (beide in array-like Formaten). Wendet diese Interpolation dann auf die x- und y-Koordinaten des Grids von oben an, und visualisiert anschließend die Ergebnisse. Stellt zur visuellen Kontrolle ebenfalls die Messwerte als Scatterplot in derselben Abbildung dar. 

In [ ]:
from scipy.interpolate import Rbf

rbfi = Rbf(data.Rechtswert, data.Hochwert, data.GWT, function='multiquadric')
di = rbfi(grid_x, grid_y)

plt.figure(3, figsize=(8,6))
c1 = plt.imshow(di.T, origin="lower", extent=extent, cmap = 'RdYlGn_r')
c2 = plt.scatter(data.Rechtswert, data.Hochwert, s=60, c=data.GWT, edgecolor='k',  cmap = 'RdYlGn_r')

plt.colorbar(c1, shrink=0.67)
plt.xlabel('Rechtswert [m]')
plt.ylabel('Hochwert [m]')
plt.show()

Da die gemessenen Grundwassertemperaturen mit gewissen Messfehlern behaftet sind, ist eine exakte Interpolation nicht unbedingt sinvoll. Mit der Angabe des Arguments "smooth=..." und Werten > 0 geht die Interpolation nicht mehr exakt durch die Messpunkte. Damit könnt Ihr das erzeugte Bild der Grundwassertemperaturen etwas glätten.

In [ ]:
rbfi = Rbf(data.Rechtswert, data.Hochwert, data.GWT, smooth=0.2)
di = rbfi(grid_x, grid_y)

plt.figure(4, figsize=(10,6))
plt.imshow(di.T, origin="lower", extent=extent, cmap = 'RdYlGn_r')
plt.scatter(data.Rechtswert, data.Hochwert, s=2, c='k')
plt.colorbar(c1, shrink=0.67)
plt.xlabel('Rechtswert [m]')
plt.ylabel('Hochwert [m]')
plt.show()

Vergleicht nun zum Abschluss die oben erzeugten Interpolationen. Welche erzeugt für euch das sinnvollste Bild der Grundwassertemperaturen?

Als zusätzliche Übung könnt Ihr z.B.:

- veschiedene Funktionen für die Interpolation mit der Radialen Basisfunktion ausprobieren, und die Unterschiede vergleichen. Die verschiedenen Möglichkeiten findet Ihr hier: https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.Rbf.html

- auch die Grundwasserhöhen in dem Datensatz interpolieren.

## Ende

### Referenzen

Menberg et al. (2013): Long-term evolution of anthropogenic heat fluxes into a subsurface urban heat island, Environ. Sci. Technol. 47(17) (2013) 9747-9755

https://colab.research.google.com/github/agile-geoscience/xlines/blob/master/notebooks/11_Gridding_map_data.ipynb